# Comparando la solución numérica con la teórica
- Óscar Alvarado
- Oscar Esquivel

In [1]:
import numpy as np

import FVM as fvm

import plotly.graph_objects as go

## Numérica

In [2]:
vol = 60
mesh = fvm.Mesh(dim = 3, volumes = (vol, vol, vol), lengths = (10, 10, 10))
mesh.tag_wall_dirichlet(direction = "B", value = 100)
mesh.tag_wall_dirichlet(direction = ["W", "E", "T", "N", "S"], value = [0,0,0,0,0])
#mesh.draw()
coef = fvm.Coefficients(mesh)
coef.set_diffusion(1_000)
sistema = fvm.EqSystem(coef)
numerical_solution = sistema.get_solution()
#sistema.plot_solution(mesh)

## Teórica (ecuación de Laplace)

In [3]:
def Aₘₙ(m,n):
    return (400/(m*n*np.pi**2))*(1-np.cos(m*np.pi))*(1-np.cos(n*np.pi))/np.sinh(np.sqrt(m**2 + n**2)*np.pi)
    
def ψ(x,y,z):
    L = 10
    n_array = range(1,100)
    m_array = range(1,100)
    solution = 0
    for n in n_array:
        for m in m_array:
            A = Aₘₙ(m,n)
            solution += A*np.sin(n*np.pi*x/L)*np.sin(m*np.pi*y/L)*np.sinh(np.sqrt(m**2 + n**2)*np.pi*z/L)
    return solution

In [4]:
X, Y, Z = np.meshgrid(mesh.coords[0], mesh.coords[1], mesh.coords[2], indexing = "ij")
analitical_solution = list(np.ravel(ψ(X,Y,Z)))

In [5]:
x,y,z = [np.array(coord) for coord in mesh.coords]
# xv, yv, zv = np.meshgrid(x, y, z, indexing='ij')

# fig = go.Figure(data = go.Scatter3d(x = xv.ravel(), y = yv.ravel(), z = zv.ravel(), mode = 'markers', 
#                                     marker = dict(color = analitical_solution, colorscale = "viridis",
#                                                 symbol = "square", size = 5, colorbar = {"bgcolor":"white"}), 
#                                     text = analitical_solution), layout = go.Layout(title = 'Solución teórica'))
# fig.show()

## Residuales

In [6]:
residual_solution = list(np.abs(np.array(numerical_solution) - np.array(analitical_solution)))
# x,y,z = [np.array(coord) for coord in mesh.coords]
# xv, yv, zv = np.meshgrid(x, y, z, indexing='ij')

# fig = go.Figure(data = go.Scatter3d(x = xv.ravel(), y = yv.ravel(), z = zv.ravel(), mode = 'markers', 
#                                     marker = dict(color = residual_solution, 
#                                                 symbol = "square", size = 5, colorscale = "OrRd",
#                                                   colorbar = {"bgcolor":"white"}), 
#                                     text = residual_solution), 
#                 layout = go.Layout(title = 'Valor absoluto de la resta de soluciones'))
# fig.show()

## Errores

In [7]:
print(np.mean(residual_solution))
print(np.var(residual_solution))

0.01288049271755015
0.002942545818688298


---